### Libraries

In [8]:
import math
import numpy as np

# import pandas 
# import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

### Constants

In [9]:
# @title Gauss-Laguerre constants
# Notation: (w_i, x_i)

GAUSS_LAGUERRE = [
    (0.458964, 0.222847),
    (0.417, 1.188932),
    (0.113373, 2.992736),
    (0.0103992, 5.775144),
    (0.000261017, 9.837467),
    (0.000000898548, 15.982874)
]

GAUSS_LEGENDRE = [
    (0.360762, -0.661209),
    (0.467914, -0.238619),
    (0.171324, -0.171324),
    (0.171324, 0.171324),
    (0.467914, 0.238619),
    (0.360762, 0.661209)
]

### PDF, CDF, plotting functions

In [10]:
# @title Weibull distribution function
# Uses Gauss-Laguerre quadratures
def f_Weibull (t, k, mu):
    denomArea = 0
    
    for node in GAUSS_LAGUERRE:
        denomArea += node[0] * (node[1]**(1/k))

    lbda = mu / denomArea

    return (k/lbda) * ((t/lbda)**(k-1)) * (math.e ** (-(t/lbda)**k))

In [11]:
# @title Integral of Weibull function
# Using Gauss-Laguerre quadratures
def cdf (t, k, mu):
    pdf_area = 0
    
    for node in GAUSS_LEGENDRE:
        pdf_area += node[0] * f_Weibull(node[1], k, mu)

    return t * 0.5 * pdf_area

In [12]:
def plot (x05, x1, x2, y05, y1, y2, xtitle, ytitle, title):
    fig = go.Figure()
    fig.add_traces([
        go.Scatter(x=x05, y=y05, mode='lines', marker = {'color' : 'blue'}, name="k = 0.5"),
        go.Scatter(x=x1, y=y1, mode='lines', marker = {'color' : 'red'}, name="k = 1"),
        go.Scatter(x=x2, y=y2, mode='lines', marker = {'color' : 'magenta'}, name="k = 2")
    ])
    fig.update_layout(
        title_text=title,
        height=1080*0.5,
        width=1920*0.6,
        xaxis_title=xtitle,
        yaxis_title=ytitle
    )
    fig.show()

### Graphs

##### PDF

In [13]:
# @title CDF values
x, y05, y1, y2 = [], [], [], []

for t in range(1, 251, 1):
    t /= 100
    x.append(t)
    y05.append(f_Weibull(t, 0.5, 2))
    y1.append(f_Weibull(t, 1, 1))
    y2.append(f_Weibull(t, 2, math.sqrt(math.pi)/2))

plot(x, x, x, y05, y1, y2, "t", "alpha", "PDF (lambda = 1)")

##### CDF

In [14]:
# @title CDF values
x, y05, y1, y2 = [], [], [], []

for t in range(1, 251, 1):
    t /= 100
    x.append(t)
    y05.append(cdf(t, 0.5, 2))
    y1.append(cdf(t, 1, 1))
    y2.append(cdf(t, 2, math.sqrt(math.pi)/2))

plot(x, x, x, y05, y1, y2, "t", "alpha", "CDF (lambda = 1)")

TypeError: Object of type complex is not JSON serializable

##### S(t)

In [ ]:
# @title Survival function
plot(x, x, x, [1-i for i in y05], [1-i for i in y1], [1-i for i in y2], "t", "alpha", "S(t), lambda = 1")

##### S^-1(alpha)

In [ ]:
# @title Inverse survival function
plot([1-i for i in y05], [1-i for i in y1], [1-i for i in y2], x, x, x, "alpha", "t", "Inverse survival (lambda = 1)")

### Root-finding problem

In [ ]:
# @title Inverse survivability
# solved as root-finding problem using bisection method
def bisection (a, b, alpha, mu, k):
    f = lambda t : 1 - cdf(t, k, mu) - alpha

    tol = 1.0e-9

    fa = f(a)
    fb = f(b)

    if fa == 0.0:
        return a
    
    if fb == 0.0:
        return b
    
    print("-> cdf(" + str(a) + ") - " + str(alpha) + " = " + str(fa))
    print("-> cdf(" + str(b) + ") - " + str(alpha) + " = " + str(fb))
    
    if np.sign(fa) == np.sign(fb):
        print("---> NO ROOT AT :", alpha)
        return None
    
    n = int (math.ceil (math.log(abs(b-a)/tol) / math.log(2.0)))

    print("---> iterations =", n)

    for i in range(n):
        # print("-> CDF(" + str(a) + ") - " + str(alpha) + " = " + str(fa))
        # print("-> CDF(" + str(b) + ") - " + str(alpha) + " = " + str(fb))

        c = 0.5 * (a + b)
        fc = f(c)

        if fc == 0.0:
            return c

        if np.sign(fa) != np.sign(fc):
            b = c
            fb = fc
        
        elif np.sign(fb) != np.sign(fc):
            a = c
            fa = fc

    print("---> returning c =", 0.5 * (a+b))
    return 0.5 * (a+b)

#### Evaluation

In [ ]:
# Lists for values storing
x_values, y_t_05, y_t_1, y_t_2 = [], [], [], []

# Average life expectancy
mu = 78

In [ ]:
# For alpha in (0,1) with 0.01 step
for alpha in range(0, 100, 1):
    alpha /= 100

    x_values.append(alpha)

    y_t_05.append(bisection(0, 100, alpha, mu, 0.5))
    y_t_1.append(bisection(0, 100, alpha, mu, 1))
    y_t_2.append(bisection(0, 100, alpha, mu, 2))

    # y_t_05.append(bisection(0, 2.5, alpha, 2, 0.5))
    # y_t_1.append(bisection(0, 2.5, alpha, 1, 1))
    # y_t_2.append(bisection(0, 2.5, alpha, math.sqrt(math.pi)/2, 2))

-> cdf(0) - 0.0 = 0.4882703528358575
-> cdf(100) - 0.0 = 0.02900891857601895
---> NO ROOT AT : 0.0
-> cdf(0) - 0.0 = 0.22961451257881582
-> cdf(100) - 0.0 = 0.06371075331964282
---> NO ROOT AT : 0.0
-> cdf(0) - 0.0 = 0.051561992146336055
-> cdf(100) - 0.0 = 0.11998951597572449
---> NO ROOT AT : 0.0
-> cdf(0) - 0.01 = 0.4782703528358575
-> cdf(100) - 0.01 = 0.019008918576018947
---> NO ROOT AT : 0.01
-> cdf(0) - 0.01 = 0.2196145125788158
-> cdf(100) - 0.01 = 0.05371075331964282
---> NO ROOT AT : 0.01
-> cdf(0) - 0.01 = 0.04156199214633605
-> cdf(100) - 0.01 = 0.1099895159757245
---> NO ROOT AT : 0.01
-> cdf(0) - 0.02 = 0.4682703528358575
-> cdf(100) - 0.02 = 0.009008918576018949
---> NO ROOT AT : 0.02
-> cdf(0) - 0.02 = 0.20961451257881583
-> cdf(100) - 0.02 = 0.04371075331964282
---> NO ROOT AT : 0.02
-> cdf(0) - 0.02 = 0.03156199214633605
-> cdf(100) - 0.02 = 0.09998951597572449
---> NO ROOT AT : 0.02
-> cdf(0) - 0.03 = 0.4582703528358575
-> cdf(100) - 0.03 = -0.0009910814239810495
--

#### Plotting

In [ ]:
# @title Inverse survivability plot
plot(x_values, x_values, x_values, y_t_05, y_t_1, y_t_2, "Probability (alpha)", "Time (t)", "Inverse survival (mu = 78)")

In [ ]:
x, y05, y1, y2 = [], [], [], []
for t in range(1, 100, 1):
    x.append(t)
    y05.append(cdf(t, 0.5, 78))
    y1.append(cdf(t, 1, 78))
    y2.append(cdf(t, 2, 78))

plot(x, x, x, [1-i for i in y05], [1-i for i in y1], [1-i for i in y2], "t", "alpha", "Survival (mu = 78)")
plot([1-i for i in y05], [1-i for i in y1], [1-i for i in y2], x, x, x, "alpha", "t", "Inverse survival (mu = 78)")